In [31]:
# All dependencies
import sys
import requests
import numpy as np
import pandas as pd
import numpy.linalg as la
import matplotlib.pyplot as plt

from os import path
from time import time
from copy import deepcopy
from mpl_toolkits.mplot3d import Axes3D as plt3

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.preprocessing import StandardScaler as SS
from sklearn.model_selection import train_test_split as tts, cross_validate, GridSearchCV as GSCV

**Commonly Used Methods**

The methods below are commonly used when running a variety of classification and regression models.

**Culling the Accidents Data**

In the `accidents.csv` file, we found that there was too much data accidents of severity 2 and three recorded, leading to nearly unreadable graphs and an unusually high accuracy.  As a result, a method was created to ensure the number of all accidents was roughly even across all severities.

In [40]:
def plot_ftrs(X, y, dimensions=2, cmap="viridis"):
    components = PCA(n_components=(dimensions)).fit(X).transform(X)
    if dimensions == 2:
        fig = plt.figure().add_subplot(111)
        scatter = fig.scatter(components[:, 0], components[:, 1], c=y, cmap=cmap)
    else:
        fig = plt.figure().add_subplot(111, projection="3d")
        scatter = fig.scatter(components[:, 0], components[:, 1], components[:, 2], c=y, cmap=cmap)
    
    plt.colorbar(scatter, ax=fig, ticks=list(np.unique(y)))
    plt.show()

def dataframe_from_csv(file_name, exclude_cols):
    cols = list(pd.read_csv(f"{file_name}.csv", nrows=1))

    return pd.read_csv(f"{file_name}.csv", usecols=[a for a in cols if a not in exclude_cols])

def get_accuracy(y_test, y_pred, use_np_mean=False):
    if use_np_mean:
        accuracy = np.mean(y_test == y_pred)
    else:
        accuracy = np.dot(y_test, y_pred) / (la.norm(y_test) * la.norm(y_pred))
    
    return float(f"{accuracy * 100:.3f}")

def cull_data(dataset, features, percent_of_even_data_to_use=1):
    accident_data = data_frames[dataset][features].dropna().values
    np.random.shuffle(accident_data)
    
    unique_labels, unique_label_counts = np.unique(accident_data[:, -1], return_counts = True)
    returning_accident_data = accident_data[accident_data[:, -1] == unique_labels[0]][:int(np.min(unique_label_counts) * percent_of_even_data_to_use), :]
    for l in range(1, len(unique_labels)):
        returning_accident_data = np.vstack((returning_accident_data, accident_data[accident_data[:, -1] == unique_labels[l]][:int(np.min(unique_label_counts) * percent_of_even_data_to_use), :]))
    
    np.random.shuffle(returning_accident_data)
    return returning_accident_data

def mult_list(val_list):
    product = len(val_list[0])

    for val in val_list:
        product *= len(val_list)

    return product

def count_iters(params):
    param_lens = [len(val) for _, val in params.items()]
    return sum(param_lens) + mult_list(param_lens)
    

def custom_cv(estimators, X, y, params_list, test_size=0.3, timed=True, verbose=False):
    if timed: start_time = time()

    ret_dict = {}
    X_train, X_test, y_train, y_test = tts(X, y, test_size=test_size)

    if len(estimators) != len(params_list):
        print("The number of estimators and corresponding parameters does not match")
        return
        
    for a in range(len(estimators)):
        estimator_name = estimators[a].__class__.__name__
        if timed: individual_time = time()
        pipe = Pipeline([("scaler", SS()), ("gscv", GSCV(estimators[a], params_list[a], verbose=1 if verbose else 0))])
        pipe.fit(X_train, y_train)

        ret_dict[estimator_name] = {
            "best_accuracy": get_accuracy(y_test, pipe.predict(X_test)),
            # "best_estimator": pipe.named_steps["gscv"].best_estimator_,
            "best_params": pipe.named_steps["gscv"].best_params_,
            "best_score": pipe.named_steps["gscv"].best_score_,
        }

        if timed: ret_dict[estimator_name]["fit_time"] = time() - individual_time
        if verbose and timed: print(f"{estimator_name} fit time: {ret_dict[estimator_name]['fit_time']:.3f} seconds")

    if timed: ret_dict["total_time"] = time() - start_time
    if verbose and timed: print(f"Total Cross Validation fit time: {ret_dict['total_time'] / 60:.3f} minutes")

    return ret_dict

def print_cv_results(cust_cv_results, precision=3):
    for key, results_dict in cust_cv_results.items():
        if not isinstance(results_dict, dict):
            continue
            
        print(f"Showing results of Grid Search and Cross Validation for: {key}")
        print(f"\tBest Params: {results_dict['best_params']}")
        print(f"\tBest Score: {results_dict['best_score']:.{precision}f}")
        print(f"\tFinal {key} Accuracy: {results_dict['best_accuracy']:.{precision}f}%")
        try:
            print(f"\t{key} Fit Time: {results_dict['fit_time']:.{precision}f} seconds\n")
        except KeyError:
            print()

    try:
        print(f"Total time to fit all classifiers: {cust_cv_results['total_time'] / 60:.{precision}f} minutes")
    except KeyError:
        pass

**Downloading of the Data Files**

We used dropbox to store the CSV files and used their urls to load them into pandas Dataframes using our custom method `dataframe_from_csv`.
To save load time, if the method first checks if the datasets are already downloading before attempting to download from the dropbox.

In [5]:
# Download data files
urls = {
    "vehicles": "https://www.dropbox.com/scl/fi/rfywugpl0vxo5mc5b7uy7/Vehicles0514.csv?rlkey=v4yygd6hc96ejfm9nt1fvadz9&dl=0",
    "casualties": "https://www.dropbox.com/scl/fi/o1ljw70bzrwmx0x8etifq/Casualties0514.csv?rlkey=iwyg1ergtazhl6mgwqk78xn5s&dl=0",
    "accidents": "https://www.dropbox.com/scl/fi/7enn6tyaavc8ch8hlib0f/Accidents0514.csv?rlkey=u316i9rln87722m1fefzfp5ba&dl=0",
    "data-guide": "https://www.dropbox.com/scl/fi/amogx0ugusmivneq6ardz/Road-Accident-Safety-Data-Guide.xls?rlkey=rnkxrf6eqjer67x2lv99tzvvv&dl=0"
}

data_frames = {}

# pd.read_excel(urls["data-guide"], sheet_name="Export Variables", engine="xlrd")

exclude_columns = ["Accident_Index", "LSOA_of_Accident_Location"]
for file_name in list(urls.keys())[:-1]:
    print(f"Loading {file_name}.csv")
    if path.exists(f"{file_name}.csv"):
        data_frames[file_name] = dataframe_from_csv(file_name, exclude_columns)
        continue
        
    direct_url = urls[file_name].replace("&dl=0", "&dl=1")
    response = requests.get(direct_url)

    with open(f"{file_name}.csv", "wb") as f:
        f.write(response.content)
        print(f"Wrote {file_name}.csv")

    data_frames[file_name] = dataframe_from_csv(file_name, exclude_columns)

print("Done!")

Loading vehicles.csv
Loading casualties.csv
Loading accidents.csv
Done!


In [6]:
# First Question: How does the weather impact the serverity of the accident?
accident_data = cull_data(
    "accidents",
    ["Weather_Conditions", "Road_Surface_Conditions", "Special_Conditions_at_Site", "Carriageway_Hazards", "Accident_Severity"], 
    0.02
)

X = accident_data[:, :-1]
y = accident_data[:, -1]

In [ ]:
plot_ftrs(X, y, dimensions=2)

In [ ]:
plt.scatter(accident_data[:, 2], accident_data[:, 3], c=y, cmap="viridis", alpha=0.75)
plt.xlabel("Special Conditions")
plt.ylabel("Carriageway hazards")
plt.show()

In [ ]:
plt.scatter(accident_data[:, 0], accident_data[:, 1], c=y, cmap="viridis", alpha=0.75)
plt.xlabel("Weather Conditions")
plt.ylabel("Road Conditions")
plt.show()

## Method
We approached testing each classifier on question 1 (Can various details about the weather be used to accuractely predict the severity of a crash?) by using Grid Search combined with Cross Validation on our culled dataset to obtain the best parameters for each classifier.

In [41]:
svc_params = {
    "kernel": ["linear", "rbf"],
    "C": [0.001, 0.01, 0.1, 1, 10],
    "gamma": [0.001, 0.01, 0.1, 1, 10],
}
rfc_params = {
    "n_estimators": [1, 10, 100, 1000]
}
knn_params = {
    "n_neighbors": [1, 2, 4, 8, 10, 20, 40, 80]
}

results = custom_cv([SVC(), RFC(), KNN()], X, y, params_list=[svc_params, rfc_params, knn_params], verbose=True)
print_cv_results(results, precision=4)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
SVC fit time: 4.119 seconds
Fitting 5 folds for each of 4 candidates, totalling 20 fits
RandomForestClassifier fit time: 6.025 seconds
Fitting 5 folds for each of 8 candidates, totalling 40 fits
KNeighborsClassifier fit time: 0.283 seconds
Total Cross Validation fit time: 0.174 minutes
Showing results of Grid Search and Cross Validation for: SVC
	Best Params: {'C': 0.1, 'gamma': 10, 'kernel': 'rbf'}
	Best Score: 0.3594
	Final SVC Accuracy: 87.4530%
	SVC Fit Time: 4.1194 seconds

Showing results of Grid Search and Cross Validation for: RandomForestClassifier
	Best Params: {'n_estimators': 1000}
	Best Score: 0.3616
	Final RandomForestClassifier Accuracy: 87.4270%
	RandomForestClassifier Fit Time: 6.0245 seconds

Showing results of Grid Search and Cross Validation for: KNeighborsClassifier
	Best Params: {'n_neighbors': 80}
	Best Score: 0.3571
	Final KNeighborsClassifier Accuracy: 87.4750%
	KNeighborsClassifier Fit Time: 0.2831 

In [11]:
print(f"Best SVM accuracy = {results['best_accuracy']:.3f}%")
print("Best SVM parameters =", results['best_params'])
print(f"Best SVM score = {results['best_score']:.3f}")

Best SVM accuracy = 89.568%
Best SVM parameters = {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Best SVM score = 0.359


In [18]:
def run_random_forest_classifier(X, y):
    
    rfc_params = {
        "n_estimators": [10, 100, 1000],
    }
    rfc_results = custom_cv(RFC(), X, y, params = rfc_params, verbose = True);
    
    # print(f"Best RFC accuracy = {rfc_results['best_accuracy']:.3f}%")
    print("Best RFC parameters = ", rfc_results['best_params'])
    print(f"Best RFC score = {rfc_results['best_score']:.3f}%")
    
    return rfc_results['best_accuracy']

In [19]:
print(f"Accuracy of Random Forest Classification: {run_random_forest_classifier(X, y)}%")

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Total Cross Validation fit time: 0.088 minutes
Best RFC parameters =  {'n_estimators': 100}
Best RFC score = 0.340%
Accuracy of Random Forest Classification: 90.015%


In [20]:
def run_knn_classifier(X, y):
    knn_params = {
        "n_neighbors": [1, 2, 4, 8, 10, 20, 40, 80],
    }
    knn_results = custom_cv(KNeighborsClassifier(), X, y, params = knn_params, verbose = True)
    
    # print(f"Best KNN accuracy = {knn_results['best_accuracy']:.3f}%")
    print("Best KNN parameters = ", knn_results['best_params'])
    print(f"Best KNN score = {knn_results['best_score']:.3f}%")
    
    return knn_results['best_accuracy']

In [21]:
print(f"Accuracy of K Nearest Neighbors Classification: {run_knn_classifier(X, y)}%")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Total Cross Validation fit time: 0.005 minutes
Best KNN parameters =  {'n_neighbors': 40}
Best KNN score = 0.360%
Accuracy of K Nearest Neighbors Classification: 87.728%
